# CIC Decimation Filter — Digital Design Flow

Digital design flow for a CIC (Cascaded Integrator-Comb) decimation filter targeting Sky130.
Demonstrates RTL simulation, waveform analysis, synthesis, static timing analysis, and place & route.

In [1]:
import sys
sys.path.insert(0, '..')

from pathlib import Path
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column

from utils.iverilog import IcarusSimulator
from utils.vcd import parse_vcd
from utils.yosys import YosysSynthesizer
from utils.openroad import OpenROADRunner

output_notebook()

RTL_DIR = Path('../src/components/digital/rtl')
SDC_FILE = RTL_DIR / 'cic_filter.sdc'

Loading BokehJS ...

## 1. RTL Simulation

The CIC filter is defined in Verilog with compile-time parameters:
- `ORDER = 3` (3 integrator + 3 comb stages)
- `DECIMATION_RATIO = 64`
- `INPUT_WIDTH = 1` (1-bit signed: sigma-delta bitstream)
- `OUTPUT_WIDTH = 16`

The testbench feeds a constant `din = 1` (which is -1 in 1-bit signed) and dumps a VCD waveform.

In [2]:
sim = IcarusSimulator()
sim_result = sim.simulate(
    verilog_files=[
        RTL_DIR / 'cic_filter.v',
        RTL_DIR / 'cic_filter_tb.v',
    ],
    top_module='cic_filter_tb',
    output_dir=Path('../work/cic_sim'),
)
print(sim_result)

Simulation successful
VCD: ../work/cic_sim/cic_filter_tb.vcd


## 2. Waveform Analysis

Parse the VCD file and plot the CIC filter output and valid strobe.

In [3]:
data = parse_vcd(sim_result.vcd_path)

# Extract signals (hierarchical names from VCD)
dout = data['cic_filter_tb.dut.dout']
valid = data['cic_filter_tb.dut.valid']

# Convert times from ps to us
t_dout = dout['times'] / 1e6
t_valid = valid['times'] / 1e6

p1 = figure(title='CIC Filter Output (dout)', x_axis_label='Time (us)',
            y_axis_label='dout', width=800, height=300)
p1.step(t_dout, dout['values'], line_width=1.5, mode='after')

p2 = figure(title='Valid Strobe', x_axis_label='Time (us)',
            y_axis_label='valid', width=800, height=200,
            x_range=p1.x_range)
p2.step(t_valid, valid['values'], line_width=1.5, mode='after')

show(column(p1, p2))

In [4]:
# Print output values at each valid pulse
valid_times = valid['times'][valid['values'] == 1]
for vt in valid_times:
    idx = np.searchsorted(dout['times'], vt, side='right') - 1
    if idx >= 0:
        print(f"  t = {vt/1e6:.1f} us  dout = {dout['values'][idx]}")

  t = 0.7 us  dout = 0
  t = 1.4 us  dout = 0
  t = 2.0 us  dout = 0
  t = 2.7 us  dout = -4964
  t = 3.3 us  dout = -26781
  t = 3.9 us  dout = -32768
  t = 4.6 us  dout = -32768
  t = 5.2 us  dout = -32768
  t = 5.9 us  dout = -32768
  t = 6.5 us  dout = -32768


## 3. Synthesis

Synthesize the CIC filter to Sky130 `sky130_fd_sc_hd` standard cells using Yosys.

In [5]:
synth = YosysSynthesizer()
synth_result = synth.synthesize(
    verilog_files=[RTL_DIR / 'cic_filter.v'],
    top_module='cic_filter',
    output_dir=Path('../work/cic_synth'),
)
print(synth_result)

Synthesis successful
Netlist: ../work/cic_synth/cic_filter_synth.v
  area_um2: 9422.7872
  cells: 1013
  wires: 781


## 4. Static Timing Analysis (post-synthesis)

STA checks all register-to-register paths against the clock constraint (100 MHz = 10 ns period).
WNS (worst negative slack) >= 0 means timing is met.

In [6]:
runner = OpenROADRunner()
sta_result = runner.run_sta(
    netlist=synth_result.netlist_path,
    sdc=SDC_FILE,
    top_module='cic_filter',
    output_dir=Path('../work/cic_sta'),
)
print(sta_result)

STA complete — WNS = 0.000 ns
  TNS = 0.000 ns
  Max frequency = 100.0 MHz
  Critical path: _1587_


## 5. Place & Route

Full P&R flow: floorplanning, placement, clock tree synthesis, routing.
Targets `sky130_fd_sc_hd` standard cells with OpenROAD. Takes ~1 minute.

In [8]:
pnr_result = runner.place_and_route(
    netlist=synth_result.netlist_path,
    sdc=SDC_FILE,
    top_module='cic_filter',
    output_dir=Path('../work/cic_pnr'),
)
print(pnr_result)
if pnr_result.sta:
    print(f'\nPost-route timing:')
    print(pnr_result.sta)

P&R successful
  DEF: ../work/cic_pnr/cic_filter.def
  GDS: ../work/cic_pnr/cic_filter.gds
  instances: 1013
  Post-route WNS = 0.000 ns

Post-route timing:
STA complete — WNS = 0.000 ns
  TNS = 0.000 ns
  Max frequency = 100.0 MHz
  Critical path: _1636_


## Summary

| Step | Tool | Output |
|------|------|--------|
| RTL simulation | Icarus Verilog | VCD waveform |
| Waveform analysis | Python (numpy, bokeh) | Plots + numerical data |
| Synthesis | Yosys | Gate-level netlist + area/cell report |
| Static timing analysis | OpenSTA (via OpenROAD) | Slack, max frequency, critical path |
| Place & route | OpenROAD | DEF layout, post-route timing |